# DO THIS :  

### *** Names: [Insert Your Names Here;      Optional: indicate preferred pronouns]***

# AND IN THE FILENAME

##### Problem Set 14     Part B
## PHYS 105A   Spring 2019

## Contents

Yet more matplotlib (widgits!), timing the execution of code, and Mandelbrot and Julia fractal sets.

** There are 9 exercises that must be completed (total in parts A and B). **

In [ ]:
%matplotlib qt

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np

from numba import jit

### Matplotlib widgets and GUIs

To create such a tool, we will need to use some graphical widgets -- helpful functions to make a "graphical user interface", or GUI.

These widgets need to be able to take input from the mouse and keyboard while the program is running. The most common way this is achieved is to have the code run in an infinite loop which is interrupted whenever input is provided. Some action is taken according to the input, and then the loop starts running again. This sort of algorithm is known as an *event loop* -- the code loops until a user event occurs.

`matplotlib` provides a number of simple widgets which automatically create an event loop for us. One can create a widget instance, and then tell the widget what function to run when something happens to the widget. Such a function is called a *callback* -- the event loop calls back to the function we give it in order to take some action before starting up again.

As a simple example, let's write a program which displays a curve whose parameters we can change using a slider widget. The callback function will change the frequency of a sine wave, replot the data, and then return.

 * import the `mathplotlib.widgets` modules.
 * create some axes and a plot object (in this case a line)
 * leave a bit of room at the bottom of our axes to put the widget
 * add a new axes in which to put the slider
 * create a callback function which gets the new value of the frequency from the slider widget, recomputes the data, and sets the line object's data to the new values.
 * create the slider object in its set of axes
 * tell the slider object the name of our callback function

In [ ]:
import matplotlib.widgets as mw          # get access to the widgets

fig, ax = plt.subplots()                 # create an axes object
plt.subplots_adjust(bottom=0.2)          # leave some room at the bottom for our slider

freq = 1                                 # code to plot a sine wave with given frequency and phase
phase = 0
x = np.linspace(0,1,100) * 4*np.pi
y = np.sin(x*freq + phase)

line, = ax.plot(x,y,'g')                 # a plot object we can modify later

def onSliderChange(dummy):               # our callback function, called when the slider changes value
    freq = freqSlider.val
    y = np.sin(x*freq + phase)
    line.set_data(x, y)
    
freqAxes = fig.add_axes( [0.3, 0.1, 0.5, 0.03] )             # add an axes object to contain the slider widget
                                                          # argument is a list [ left, bottom, width, height ]

freqSlider = mw.Slider(freqAxes, 'frequency', valmin=0, valmax=4, valinit=1)  # the widget
freqSlider.on_changed(onSliderChange)                                      # tell the widget about our callback

#### Exercise 7

Copy the code in the previous cell to the following cell.
Modify it to add another slider for the phase:

* Make another axis object to contain the new slider; adjust its location so it fits on the page beneath the frequency slider.
* Make a new slider called 'Phase' with limits from $-\pi$ to $\pi$ and initial value $0$.
* Change the callback function onSliderChanged to get the new value of the phase from the new slider's value and use it in recalculating the data. There is no need to write a new callback function.
* Connect the new slider object to the callback function you just modified

We can catch characters typed on the keyboard -- *keypress events* -- by connecting a "key_press_event" to a callback function which takes an event as an argument.
The event object contains a variety of data. The most useful being:

    event.key       # the key which was pressed
    event.xdata     # the mouse x-position when the key was pressed
    event.ydata     # the mouse y-position when the key was pressed

In the following, note the sequence of events that happens when one presses the 'r' key:

* Pressing the 'r' key stops the event loop and calls the onKeyPressed callback function.
* In the onKeyPressed function resets the sliders to their initial values.
* Since the sliders' values changed, they in turn call the onSliderChanged callback function.
* The onSliderChanged function uses the new values returned by the slider objects to reset the data in line.
* When onSliderChanged returns, control returns to onKeyPressed.
* When onKeyPressed returns, the plot is refreshed and the event loop resumes

When you press 'r', onSliderChange only is called if one or more sliders have in fact changed. If both are changed,
the onSliderChanged is called *twice* -- once for each slider.

One more thing to note. When a graphical widget callback function returns to the widget that called it, the plot is refreshed (similar to calling plt.pause during an animation). Because a key_pressed_event is not a graphical widget (it does not involve the plot), when control returns it does not refresh the plot. If you do something in a key_pressed_event callback which is supposed to change the plot, you must call fig.canvas.draw() yourself to make the plot refresh.

In the following example, because onSliderChanged is a graphical widget callback, the plot gets refreshed when it returns, and we therefore don't have to call fig.canvas.draw() at the end of onKeyPressed.

In [ ]:
fig, ax = plt.subplots()                      # create an axes
plt.subplots_adjust(bottom=0.2)               # leave some room at the bottom

freq = 1
phase = 0
x = np.linspace(0,1,100) * 4*np.pi
y = np.sin(x*freq + phase)

line, = ax.plot(x,y,'g')

def onSliderChange(dummy):                    # callback for slider changes
    print("        entering onSliderChange")
    freq = fslider.val
    phase = pslider.val
    y = np.sin(x*freq + phase)
    line.set_data(x, y)
    print("        finished onSliderChange")

def onKeyPressed(event):                      # callback for key_press_event
    print(f"Entering onKeyPressed: you pressed key: {event.key}")
    
    if event.key in ['R','r']:                # If key is 'r' or 'R', reset the sliders to their default values
        print("    resetting slider values")
        fslider.reset()
        pslider.reset()
    
    print("finished onKeyPressed")
    
saxes = fig.add_axes( [0.3, 0.1, 0.5, 0.03] )
fslider = mw.Slider(saxes, 'frequency', 0, 4, valinit=1)
fslider.on_changed(onSliderChange)

paxes = fig.add_axes( [0.3, 0.05, 0.5, 0.03] )
pslider = mw.Slider(paxes, 'phase', -np.pi, np.pi, valinit=0)
pslider.on_changed(onSliderChange)

plt.connect("key_press_event", onKeyPressed)

Another useful widget allows the user to select a rectangular region in some axes object, and then calls a callback function with the bounding coordinates (the extent) of the region selected. This is the RectangleSelector widget.

Note that click and release are not really that! Click contains the more-negative values and release the more positive values of both x and y coordinates.

In [ ]:
fig, ax = plt.subplots()                             # create an axes

r = np.linspace( 0, 10, 1000 )                       # psychedelic plot to look at
theta = r**2
x = r*np.cos(theta)
y = r*np.sin(theta)
ax.plot(  x,  y, 'g')
ax.plot( -x, -y, 'r')


def callbackRectangle( click, release ):
    print( f"button {click.button} pressed" )
    print( f"button {release.button} released" )
    extent = [ click.xdata, release.xdata, click.ydata, release.ydata ]
    print( f"box extent is {extent}") 
  
rs = mw.RectangleSelector( ax,                        # the axes to attach to
                           callbackRectangle,         # the callback function
                           drawtype='box',            # draw a box when selecting a region
                           button=[1, 3],             # allow us to use left or right mouse button
                           minspanx=5, minspany=5,    # don't accept a box of fewer than 5 pixels
                           spancoords='pixels' )      # units for above


#### Exercise 8

Copy the previous code cell to the following. In the callback funtion, reset the limits on the plot axes to reflect the extent returned in click and release. In effect, you are creating a program which "zooms in" on the data in the plot.

We are now ready to use widgets to create a Mandelbrot set explorer GUI.

Here is our Mandelbrot/Julia set creation code from the previous part of this notebook:

In [ ]:
@jit
def onePointIterator(constValue, z0, itmax, R, llR):
    z = z0
    for n in range(1,itmax):
        az = np.abs(z)
        if az > R:
            return n - np.log( np.log( az ))/np.log(2) + llR
        z = z*z + constValue
    return 0

@jit
def mandelbrotSet(extent, itmax, npix):
    
    R = 2.0**40
    llR = np.log( np.log( R ) ) / np.log(2)
    
    cr = np.linspace( extent[0], extent[1], npix )
    ci = np.linspace( extent[2], extent[3], npix )

    M = np.zeros( (npix,npix), dtype=np.float32)

    for i in range(npix):
        for j in range(npix):
            c = cr[i]+ci[j]*1j
            z0 = 0.0 + 0.0*1j
            M[j,i] = onePointIterator( c, z0, itmax, R, llR )

    return M

@jit
def juliaSet(c, extent, itmax, npix):
    
    R = 2.0**40
    llR = np.log( np.log( R ) ) / np.log(2)
    
    cr = np.linspace( extent[0], extent[1], npix )
    ci = np.linspace( extent[2], extent[3], npix )

    M = np.zeros( (npix,npix), dtype=np.float32)

    for i in range(npix):
        for j in range(npix):
            z0 = cr[i]+ci[j]*1j
            M[j,i] = onePointIterator( c, z0, itmax, R, llR )

    return M


The first thing we want to do is to be able to select a rectangular region and then recompute the Mandelbrot set on that region, allowing us to "zoom in" on a region of the set.

Copying the example above, let's replace the line-plotting code with our Mandelbrot-plotting code

In [ ]:
def callbackRectangle( click, release ):
    extent = [ click.xdata, release.xdata, click.ydata, release.ydata ]     # get the new extent
    M = mandelbrotSet( extent, itmax, npix )                                # create the data
    mimage.set_array(M)                                                     # give imshow object the new data
    mimage.set_extent(extent)                                               #    and new extent

npix = 512
itmax = 80
extent = [ -2.5, 1.5, -1.5,1.5 ]
M = mandelbrotSet( extent, itmax, npix )

fig, ax = plt.subplots()

norm = colors.PowerNorm( 0.3 )
mimage = ax.imshow( M, cmap="gnuplot2", extent=extent, norm=norm, origin='lower' )
  
rs = mw.RectangleSelector( ax, callbackRectangle, drawtype='box', button=[1, 3], 
                           minspanx=5, minspany=5, spancoords='pixels'           )

This allows us to zoom in, but what if we want to go back? 

The easiest thing to do is to make pressing 'r' reset the extent to the original one.

Note that after you press 'r', you may have to wait a bit for mandelbrotSet to recalculate.

Note also that, since keypresses are not, technically, widgets, we need to call fig.canvas.draw() to
replot the changed data in the plot window.

In [ ]:
def callbackRectangle( click, release ):
    extent = [ click.xdata, release.xdata, click.ydata, release.ydata ]     # get the new extent
    M = mandelbrotSet( extent, itmax, npix )                                # create the data
    mimage.set_array(M)                                                     # give imshow object the new data
    mimage.set_extent(extent)                                               #    and new extent

def onKeyPressed(event):
    
    if event.key in ['R', 'r']:
        M = mandelbrotSet( extent0, itmax, npix )                           # create the data
        mimage.set_array( M )                                               # give imshow object the new data
        mimage.set_extent( extent0 )                                        #    and new extent
        fig.canvas.draw()                                                   # need to redraw the canvas since this
                                                                            #     wasn't a graphics widget...

npix = 512
itmax = 80
extent0 = [ -2.5, 1.5, -1.5,1.5 ]                                           # call this extent0 so we can refer to it
M = mandelbrotSet( extent0, itmax, npix )                                   #   when doing a reset

fig, ax = plt.subplots()

norm = colors.PowerNorm( 0.3 )
mimage = ax.imshow( M, cmap="gnuplot2", extent=extent, norm=norm, origin='lower' )
  
rs = mw.RectangleSelector( ax, callbackRectangle, drawtype='box', button=[1, 3], 
                           minspanx=5, minspany=5, spancoords='pixels'           )

plt.connect("key_press_event", onKeyPressed)

Now, let's add drawing the Julia set, using as its iteration constant c wherever the mouse pointer happens to be.
Remember, this was at [event.xdata, event.ydata].

In [ ]:
def callbackRectangle( click, release ):
    extent = [ click.xdata, release.xdata, click.ydata, release.ydata ]
    M = mandelbrotSet( extent, itmax, npix )
    mimage.set_array( M )
    mimage.set_extent( extent )

def onKeyPressed( event ):
    
    if event.key in ['R', 'r']:
        M = mandelbrotSet( extent0, itmax, npix )                           # create the data
        mimage.set_array( M )                                               # give imshow object the new data
        mimage.set_extent( extent0 )                                        #    and new extent
        fig.canvas.draw()
        
    if event.key in ['J', 'j']:
        c = event.xdata + event.ydata * 1j
        J = juliaSet(c, juliaExtent, itmax, npix)
        jimage.set_array( J ) 
        jimage.set_extent( juliaExtent )
        fig.canvas.draw() 
        
npix = 512
itmax = 80
extent0 = [ -2.5, 1.5, -1.5,1.5 ]
M = mandelbrotSet( extent0, itmax, npix )
juliaExtent = [ -1.5, 1.5, -1.5, 1.5 ]
J = juliaSet(0, juliaExtent, itmax, npix)

fig, ax = plt.subplots(nrows=1, ncols=2)                                    # create two subplots:
                                                                            #   ax[0] for Mandelbrot set
                                                                            #   ax[1] for Julia set
norm = colors.PowerNorm( 0.3 )
mimage = ax[0].imshow( M, cmap="gnuplot2",      extent=extent, norm=norm, origin='lower' )
jimage = ax[1].imshow( J, cmap="gnuplot2", extent=juliaExtent, norm=norm, origin='lower')

rs = mw.RectangleSelector( ax[0], callbackRectangle, drawtype='box', button=[1, 3],   # attach to Mandelbrot subplot
                           minspanx=5, minspany=5, spancoords='pixels'              )

plt.connect("key_press_event", onKeyPressed)

Once we zoom in, we should increase the number of iterations to determine the correct rate of divergence.

Let's add a slider to adjust itmax, and while we are at it, add one to change gamma as well.

For both, we'll need to add yet another axes object, a slider widget, and a callback function, and then connect the callback to the slider object as we did in the slider example above.

Rather than add all of the code for this to each callback, let's "refactor" our code. 

We'll make one function, updateFigure(), to call of the plotting functions in one place. The callback functions will simply change the data as necessary.

We will need to pay attantion to what are local and what are global variables. In particular, to write to a global
variable within a function, the variable must be declared global. Remember, however, that global *arrays* don't need to be declared global if we are only changing their contents, not making new arrays. This is for the same reasons we don't need to return an array from a function as we have discussed in class.

In [ ]:
def updateFigure( ):
    """
    Update the two subplots and their associated plot object properties
    """
    mimage.set_array( M )                            # Set Mandelbrot image
    mimage.set_clim( vmin=M.min(), vmax=M.max() )    # Rescale Mandelbrot image
    mimage.set_extent( extent )                      # Set extent for limits on Mandelbrot plot
    norm = colors.PowerNorm( gamma )                 # Get a new nomalization object as gamma correction
    mimage.set_norm( norm )                          # Set normalization object

    jimage.set_array( J )                            # Set Julia image
    jimage.set_clim( vmin=J.min(), vmax=J.max() )    # Rescale Julia image colors
    jimage.set_extent( juliaExtent )                 # Set extent for limits on Julia plot
    jimage.set_norm( norm )                          # Get a new nomalization object as gamma correction

    fig.canvas.draw()                                # Required for colormap to update in plot


def callbackRectangle(click, release):                                      # get the new extent to plot
    extent[:] = [ click.xdata, release.xdata, click.ydata, release.ydata ]  # put it in the GLOBAL extent variable
    M[:,:] = mandelbrotSet(extent, itmax, npix)                             # create the data, put it in the global M

    updateFigure()

def onKeyPressed(event):
    global extent, juliaC, M, J
    
    if event.key in ['R', 'r']:
        extent[:] = extent0                                                 # Set global extent, M, and J
        M[:,:] = mandelbrotSet(extent, itmax, npix)
        
    elif event.key in ['J','j']:
        juliaC = event.xdata + event.ydata * 1j
        J[:,:] = juliaSet(juliaC, juliaExtent, itmax, npix)

    updateFigure()

def onGamSliderChanged(val):
    global gamma                                                            # need to declare gamma a global to set it
    gamma = gamSlider.val                                                   # get the new gamma value from the slider
    updateFigure()

def onItSliderChanged(val):
    global itmax                                                            # need to declare itmax a global to set it
    itmax = int(itSlider.val)                                               # sliders only work with floats; 
                                                                            #   we need to make this an int
    M[:,:] = mandelbrotSet(extent, itmax, npix)                             # create the data, put it in old M and J
    J[:,:] = juliaSet(juliaC, juliaExtent, itmax, npix)
    
    updateFigure()
    
npix = 512
itmax = 128
gamma = 0.3
extent0 = [-2.5, 1.5, -1.5,1.5]
juliaC = 0.0 + 0.0*1j
juliaExtent = [ -1.5, 1.5, -1.5, 1.5 ]

extent = extent0.copy()                                # need to make a deep copy so as not to change extent0

M = mandelbrotSet(extent0, itmax, npix)
J = juliaSet(juliaC, juliaExtent, itmax, npix)

fig, ax = plt.subplots(nrows=1, ncols=2)
plt.subplots_adjust(bottom=0.2)                        # leave some room for sliders at the bottom

norm = colors.PowerNorm(0.3)
mimage = ax[0].imshow( M, cmap="gnuplot2", extent=extent,      norm=norm, origin='lower')
jimage = ax[1].imshow( J, cmap="gnuplot2", extent=juliaExtent, norm=norm, origin='lower')
    
rs = mw.RectangleSelector(ax[0], callbackRectangle,               # add this to Mandelbrot subplot
                                        drawtype='box',
                                       button=[1, 3],             # don't use middle button
                                       minspanx=5, minspany=5,    # don't accept a box of fewer than 5 pixels
                                       spancoords='pixels')

plt.connect("key_press_event", onKeyPressed)

gamSliderAxes  = fig.add_axes([0.6, 0.05, 0.3, 0.01])            # slider for gamma correction
gamSlider = mw.Slider(gamSliderAxes, 'gamma', 0, 1, valinit=0.3)
gamSlider.on_changed(onGamSliderChanged)

itSliderAxes  = fig.add_axes([0.6, 0.025, 0.3, 0.01])            # slider for gamma correction
itSlider = mw.Slider(itSliderAxes, 'itmax', 0, 4096, valinit=256)
itSlider.on_changed(onItSliderChanged)

#### Exercise 9

A radio button is a set of labeled buttons in a box. 

The exercise is to copy the Mandelbrot viewer above to a new cell, and add a colormap radio button to it, following
this example:

In [ ]:

colorMaps = [ 'gnuplot2','hot','nipy_spectral', 'viridis' ]

def updateFigure():
    image.set_cmap(cmap)

    fig.canvas.draw()
    
def onCmapChanged( value ):
    global cmap                                                # need to declare cmap global since we are setting it
    cmap = value                                               # set name of new colormap
    updateFigure()

    
# some data in an array...
x = np.linspace(0,1,300) * 2 * np.pi
X, Y = np.meshgrid( x, x )
M = np.sin(X) * np.cos(2*Y) 

cmap = colorMaps[0]
    
fig, ax = plt.subplots()
plt.subplots_adjust(bottom=0.2)                        # leave some room for bottons at the bottom

image = ax.imshow(M, cmap=cmap, origin='lower')

# Radio button for colormap selection
cmapRadioAxes = fig.add_axes( [0.0, 0.01,.2,0.15] )
cmapRadio     = mw.RadioButtons( cmapRadioAxes, colorMaps )
cmapRadio.on_clicked( onCmapChanged )
